데이터 전처리

In [13]:
import pandas as pd

# Load dataset
file_path = 'kma_2022_gwangju_noNULL_noDuplicated.csv'  # 파일 경로를 지정하세요.
data = pd.read_csv(file_path)

# Convert 'TM' column to datetime format
data['TM'] = pd.to_datetime(data['TM'], format='%Y%m%d%H%M')

# Select relevant columns for prediction
selected_columns = ['STN','TM', 'WD', 'WS', 'PA', 'PS', 'TA', 'TD', 'HM', 'CA_TOT', 'CA_MID', 'VS', 'TS', 'TE_005', 'TE_01', 'TE_02','TE_03', 'PV', 'PM10']
processed_data = data[selected_columns]

# Sort by time
processed_data = processed_data.sort_values('TM').reset_index(drop=True)

BASE MODEL : ARIMA 모델

In [14]:
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

arima_data = processed_data[['TM', 'PM10']].set_index('TM')

train_size = int(len(arima_data) * 0.7)
val_size = int(len(arima_data) * 0.15)

train_arima = arima_data.iloc[:train_size]['PM10']
y_val = arima_data.iloc[train_size:train_size + val_size]['PM10']
y_test = arima_data.iloc[train_size + val_size:]['PM10']
arima_order = (5, 1, 0)  # (p, d, q) 값 조정 가능
arima_model = ARIMA(train_arima, order=arima_order)
arima_fit = arima_model.fit()

# 예측
forecast_val = arima_fit.forecast(steps=len(y_val))
forecast_test = arima_fit.forecast(steps=len(y_test))

# 평가
val_rmse = np.sqrt(mean_squared_error(y_val, forecast_val))
val_mae = mean_absolute_error(y_val, forecast_val)
test_rmse = np.sqrt(mean_squared_error(y_test, forecast_test))
test_mae = mean_absolute_error(y_test, forecast_test)

print(f"ARIMA Validation RMSE: {val_rmse}, MAE: {val_mae}")
print(f"ARIMA Test RMSE: {test_rmse}, MAE: {test_mae}")


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


ARIMA Validation RMSE: 40.444368449776945, MAE: 18.396029724321956
ARIMA Test RMSE: 32.62832307024315, MAE: 19.10128552413279


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


MLP model

In [15]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Features (X) and target (y)
X = processed_data.drop(columns=['TM', 'PM10'])
y = processed_data['PM10']

# Split into training, validation, and test sets
train_size = 0.7
X_train, X_temp, y_train, y_temp = train_test_split(X, y, train_size=train_size, shuffle=False)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, shuffle=False)


# 데이터 정규화
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# MLP 모델 설계
mlp_model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)  # Output layer
])

mlp_model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# 학습
history = mlp_model.fit(
    X_train_scaled, y_train,
    validation_data=(X_val_scaled, y_val),
    epochs=50,
    batch_size=32,
    verbose=1
)

# 평가
train_loss = history.history['loss'][-1]  # 마지막 epoch의 train loss
val_loss, val_mae = mlp_model.evaluate(X_val_scaled, y_val, verbose=0)
test_loss, test_mae = mlp_model.evaluate(X_test_scaled, y_test, verbose=0)

print(f"MLP Train Loss (RMSE): {np.sqrt(train_loss)}")
print(f"MLP Validation Loss (RMSE): {np.sqrt(val_loss)}, MAE: {val_mae}")
print(f"MLP Test Loss (RMSE): {np.sqrt(test_loss)}, MAE: {test_mae}")

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 710.7253 - mae: 18.7008 - val_loss: 1225.0396 - val_mae: 14.8091
Epoch 2/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 251.9677 - mae: 10.8915 - val_loss: 1209.5156 - val_mae: 13.6332
Epoch 3/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 244.9678 - mae: 10.4979 - val_loss: 1191.9388 - val_mae: 14.3104
Epoch 4/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 227.9834 - mae: 10.2810 - val_loss: 1203.8357 - val_mae: 14.4899
Epoch 5/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 219.3726 - mae: 9.9146 - val_loss: 1194.8020 - val_mae: 14.6069
Epoch 6/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 200.9199 - mae: 9.9537 - val_loss: 1187.9938 - val_mae: 14.0758
Epoch 7/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 215.7850 - mae: 9.9049 - val_loss: 1203.2819 - val_mae: 14.5520
Epoch 8/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 201.3733 - mae: 9.6042 - val_loss: 1184.4318 - val_mae: 14.2856
Epoc

In [16]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Features (X) and target (y)
X = processed_data.drop(columns=['TM', 'PM10'])
y = processed_data['PM10']

# Split into training, validation, and test sets
train_size = 0.7
X_train, X_temp, y_train, y_temp = train_test_split(X, y, train_size=train_size, shuffle=False)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, shuffle=False)


# 데이터 정규화
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# MLP 모델 설계
mlp_model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)  # Output layer
])

mlp_model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# 학습
history = mlp_model.fit(
    X_train_scaled, y_train,
    validation_data=(X_val_scaled, y_val),
    epochs=200,
    batch_size=32,
    verbose=1
)

# 평가
train_loss = history.history['loss'][-1]  # 마지막 epoch의 train loss
val_loss, val_mae = mlp_model.evaluate(X_val_scaled, y_val, verbose=0)
test_loss, test_mae = mlp_model.evaluate(X_test_scaled, y_test, verbose=0)

print(f"MLP Train Loss (RMSE): {np.sqrt(train_loss)}")
print(f"MLP Validation Loss (RMSE): {np.sqrt(val_loss)}, MAE: {val_mae}")
print(f"MLP Test Loss (RMSE): {np.sqrt(test_loss)}, MAE: {test_mae}")

Epoch 1/200


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


187/187 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 641.4883 - mae: 17.5288 - val_loss: 1195.7706 - val_mae: 14.2695
Epoch 2/200
187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 255.3756 - mae: 10.9423 - val_loss: 1210.2181 - val_mae: 14.1312
Epoch 3/200
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 229.6860 - mae: 10.5193 - val_loss: 1196.2682 - val_mae: 14.8311
Epoch 4/200
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 245.0534 - mae: 10.3402 - val_loss: 1186.1312 - val_mae: 14.4751
Epoch 5/200
187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 215.1015 - mae: 10.1141 - val_loss: 1198.4506 - val_mae: 14.1763
Epoch 6/200
187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 205.1563 - mae: 9.8368 - val_loss: 1188.7345 - val_mae: 13.7048
Epoch 7/200
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 194.1025 - mae: 9.7389 - val_loss: 1181.1962 - val_mae: 14.1548
Epoch 8/200
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 185.5320 - mae: 9.4610 - val_loss: 1181.7766 - val_mae: 14.0374
Epoch 9